In [1]:
import numpy as np
import SWMM_ENV
import Buffer
import tensorflow as tf
import PPO as PPO
import Rainfall_data as RD

import datetime
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [1]:
import tensorflow

In [4]:
tf.compat.v1.reset_default_graph()
env_params={
    'orf':'chaohu_GI_init',
    'parm':'./states_yaml/chaohu',
    'GI':True,
    'advance_seconds':300,
    'kf':0.5,
    'kc':0.5,
}
env=SWMM_ENV.SWMM_ENV(env_params)

raindatae = np.load('./training_rainfall/training_raindata.npy').tolist()
raindataw = np.load('./training_rainfall/training_raindata.npy').tolist()
temeast = np.load('./test_rainfall/east.npy').tolist()
temwest = np.load('./test_rainfall/west.npy').tolist()
#raindata2top = np.load('./training_rainfall/2top.npy').tolist()
#raindata3top = np.load('./training_rainfall/3top.npy').tolist()

raindatae=temeast+raindatae
raindataw=temwest+raindataw

agent_params={
    'state_dim':len(env.config['states']),
    'action_dim':int(2**len(env.config['action_assets'])),
    'actornet_layer':[50,50,50,50],
    'criticnet_layer':[50,50,50,50],
    
    'bound_low':0,
    'bound_high':1,
    
    'clip_ratio':0.01,
    'target_kl':0.03,
    'lam':0.01,
    
    'policy_learning_rate':0.001,
    'value_learning_rate':0.001,
    'train_policy_iterations':20,
    'train_value_iterations':20,
    
    'num_rain':50,
    
    'training_step':500,
    'gamma':0.3,
    'epsilon':0.99,
    'ep_min':1e-50,
    'ep_decay':0.1
}


Train=True
init_train=True

model = PPO.PPO(agent_params)
if init_train:
    model.critic.save_weights('./model/PPOcritic.h5')
    model.actor.save_weights('./model/PPOactor.h5')
model.load_model('./model/')

In [5]:
###############################################################################
# Train
###############################################################################
    
def interact(i,ep):   
    env=SWMM_ENV.SWMM_ENV(env_params)
    tem_model = PPO.PPO(agent_params)
    tem_model.load_model('./model/')
    tem_model.params['epsilon']=ep
    s,a,r,vt,lo = [],[],[],[],[]
    observation, episode_return, episode_length = env.reset(raindatae[i],raindataw[i],i,True), 0, 0
    
    done = False
    while not done:
        # Get the logits, action, and take one step in the environment
        observation = np.array(observation).reshape(1, -1)
        logits, action = PPO.sample_action(observation,tem_model,True)
        at = tem_model.action_table[int(action[0].numpy())].tolist()
        observation_new, reward, flooding,CSO,Res_tn,DRes_tn,main_flow,done = env.step(at)
        episode_return += reward
        episode_length += 1

        # Get the value and log-probability of the action
        value_t = tem_model.critic(observation)
        logprobability_t = PPO.logprobabilities(logits, action, tem_model.params['action_dim'])
        
        # Store obs, act, rew, v_t, logp_pi_t
        # buffer.store(observation, action, reward, value_t, logprobability_t)
        s.append(observation)
        a.append(action)
        r.append(reward)
        vt.append(value_t)
        lo.append(logprobability_t)
        
        # Update the observation
        observation = observation_new
    # Finish trajectory if reached to a terminal state
    last_value = 0 if done else tem_model.critic(observation.reshape(1, -1))
    return s,a,r,vt,lo,last_value,episode_return,episode_length

In [6]:
if Train:
    tf.config.experimental_run_functions_eagerly(True)

    # main training process   
    history = {'episode': [], 'Batch_reward': [], 'Episode_reward': [], 'Loss': []}
    
    # Iterate over the number of epochs
    for epoch in range(model.params['training_step']):
        # Initialize the sum of the returns, lengths and number of episodes for each epoch
        sum_return = 0
        sum_length = 0
        num_episodes = 0
        
        # Initialize the buffer
        buffer = Buffer.Buffer(model.params['state_dim'], int(len(raindatae[0])*model.params['num_rain']))
        
        # Iterate over the steps of each epoch
        # Parallel method in joblib
        res = Parallel(n_jobs=10)(delayed(interact)(i,model.params['epsilon']) for i in range(model.params['num_rain'])) 
        
        for i in range(model.params['num_rain']):
            #s, a, r, vt, lo, lastvalue in buffer
            for o,a,r,vt,lo in zip(res[i][0],res[i][1],res[i][2],res[i][3],res[i][4]):
                buffer.store(o,a,r,vt,lo)
            buffer.finish_trajectory(res[i][5])
            sum_return += res[i][6]
            sum_length += res[i][7]
            num_episodes += 1
        
        
        # Get values from the buffer
        (
            observation_buffer,
            action_buffer,
            advantage_buffer,
            return_buffer,
            logprobability_buffer,
        ) = buffer.get()
    
        # Update the policy and implement early stopping using KL divergence
        for _ in range(model.params['train_policy_iterations']):
            kl = PPO.train_policy(observation_buffer, action_buffer, logprobability_buffer, advantage_buffer, model)
            #if kl > 1.5 * target_kl:
                ## Early Stopping
                #break
    
        # Update the value function
        for _ in range(model.params['train_value_iterations']):
            PPO.train_value_function(observation_buffer, return_buffer, model)
        
        #model.critic.save_weights('./model/PPOcritic_'+str(epoch)+'.h5')
        #model.actor.save_weights('./model/PPOactor_'+str(epoch)+'.h5')
        model.critic.save_weights('./model/PPOcritic.h5')
        model.actor.save_weights('./model/PPOactor.h5')
        # log training results
        history['episode'].append(epoch)
        history['Episode_reward'].append(sum_return)
        # reduce the epsilon egreedy and save training log
        if model.params['epsilon'] >= model.params['ep_min'] and epoch % 3 == 0:
            model.params['epsilon'] *= model.params['ep_decay']

        # Print mean return and length for each epoch
        print(
            f" Epoch: {epoch + 1}. Return: {sum_return}. Mean Length: {sum_length / num_episodes}"
        )
        
        np.save('./Results/Train2.npy',history)
    
    # plot
    plt.figure()
    plt.plot(history['Episode_reward'])
    plt.savefig('./Results/Train2.tif')

    
###############################################################################
# end Train
###############################################################################

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


exception calling callback for <Future at 0x1fae4dd7f40 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\joblib\externals\loky\process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "e:\anaconda3\envs\tensorflow-cpu\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\joblib\externals\cloudpickle\cloudpickle.py", line 649, in subimport
    __import__(name)
  File "e:\Github\Text13_DRL_resilience\v3\Step4_RTC\PPO\PPO.py", line 10, in <module>
    import tensorflow as tf
  File "e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\tensorflow\__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "e:\anaconda3\envs\tensorflow-cpu\lib\site

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.